In [5]:
from neo4j import GraphDatabase

class Local:
    def __init__(self,nombre,ubicacion):
        self.nombre=nombre
        self.ubicacion=ubicacion

class Resultado:
    def __init__(self,nombre,similitud):
        self.nombre=nombre
        self.similitud=similitud

class Comida:
    def __init__(self,nombre):
        self.nombre=nombre

def get_ed(tx,ciudad):
    comunidad = []
    result = tx.run("MATCH (l1:Local {lugar:$ciudad}) "
                    "MATCH (l2:Local) WHERE l1=l2 "
                    "RETURN l2.nombre as Nombre, l2.ubicacion as Ubicacion, gds.alpha.linkprediction.sameCommunity(l1,l2,'lugar') "
                    "AS Score ORDER BY Score DESC",ciudad=ciudad)
    for record in result:
        p1=Local(record["Nombre"],record["Ubicacion"])
        comunidad.append(p1)
    return comunidad
        
def getLocales(ciudad):
    text =""
    uri = "bolt://35.239.4.175:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j"),encrypted=False)
    with driver.session() as session:
        comunidad= session.read_transaction(get_ed,ciudad)
        for i in range(len(comunidad)):
            text += comunidad[i].nombre+" "+comunidad[i].ubicacion+" \n"
            #print(comunidad[i].labels)
            #text+=comunidad[i]["nombre"]+" "+comunidad[i]["correo"]+"\n"
            #print(comunidad[i]["name"]," -> ",comunidad[i]["apellido"]," -> ", comunidad[i]["edad"])
    driver.close()
    return text

def get_ed2(tx,vector):
    comunidad = []
    result = tx.run("MATCH (n:comida) "
                    "RETURN n.nombre as Comida, " 
                    "gds.alpha.similarity.cosine(n.indoNutricional, $vector) "
                    "AS similarity order by similarity desc limit 3",vector=vector)
    for record in result:
        r1=Resultado(record["Comida"],record["similarity"])
        comunidad.append(r1)
    return comunidad
def getComidas(vector):
    text =""
    uri = "bolt://35.239.4.175:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j"),encrypted=False)
    with driver.session() as session:
        comunidad= session.read_transaction(get_ed2,vector)
        for i in range(len(comunidad)):
            text += str(comunidad[i].nombre)+" "+str(int(round(comunidad[i].similitud,2)*100))+"% \n"
            #print(comunidad[i].labels)
            #text+=comunidad[i]["nombre"]+" "+comunidad[i]["correo"]+"\n"
            #print(comunidad[i]["name"]," -> ",comunidad[i]["apellido"]," -> ", comunidad[i]["edad"])
    driver.close()
    return text

def get_ed3(tx,tipo):
    comunidad = []
    result = tx.run("MATCH(n:comida) WHERE n.tipo=$tipo return n.nombre as nombre",tipo=tipo)
    for record in result:
        r1=Comida(record["nombre"])
        comunidad.append(r1)
    return comunidad

def getComis(tipo):
    text =""
    uri = "bolt://35.239.4.175:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j"),encrypted=False)
    with driver.session() as session:
        comunidad= session.read_transaction(get_ed3,tipo)
        for i in range(len(comunidad)):
            text += str(comunidad[i].nombre)+" \n"
            #print(comunidad[i].labels)
            #text+=comunidad[i]["nombre"]+" "+comunidad[i]["correo"]+"\n"
            #print(comunidad[i]["name"]," -> ",comunidad[i]["apellido"]," -> ", comunidad[i]["edad"])
    driver.close()
    return text

def get_ed4(tx,numero,por):
    comunidad = []
    result = tx.run("MATCH(n:comida) return n.nombre as nombre, n.indoNutricional["+str(numero)+"] as f order by f "+por+" limit 3")
    for record in result:
        r1=Comida(record["nombre"])
        comunidad.append(r1)
    return comunidad

def getRecomendacion(numero,por):
    text =""
    uri = "bolt://35.239.4.175:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j"),encrypted=False)
    with driver.session() as session:
        comunidad= session.read_transaction(get_ed4,numero,por)
        for i in range(len(comunidad)):
            text += str(comunidad[i].nombre)+ " \n"
            #print(comunidad[i].labels)
            #text+=comunidad[i]["nombre"]+" "+comunidad[i]["correo"]+"\n"
            #print(comunidad[i]["name"]," -> ",comunidad[i]["apellido"]," -> ", comunidad[i]["edad"])
    driver.close()
    return text

In [6]:
import json
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_cloud_sdk_core.api_exception import ApiException
from ibm_watson import LanguageTranslatorV3
from ibm_watson import PersonalityInsightsV3
from ibm_watson import TextToSpeechV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('vh0j48-6QDFVQy6-qehepHiOX_8Dc8_3anwEFFJ-7CRr')
language_translator = LanguageTranslatorV3(
    version='2018-05-01',
    authenticator=authenticator
)

language_translator.set_service_url('https://api.us-south.language-translator.watson.cloud.ibm.com/instances/590a49d1-ee08-4611-9f75-62ae74ee0d7c')

authenticator = IAMAuthenticator('RzHAgFvAbd4YJn3-21N8p6YehshWsMeLdlGqa3hmSmuR')
assistant = AssistantV2(
    version='2020-04-01',
    authenticator=authenticator
)

assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com/instances/9276aae1-9178-4219-ac80-ec84e0124427')
assistant.set_disable_ssl_verification(False)

session = assistant.create_session( assistant_id='ac28a7b9-f962-4376-a98c-1b0a8c5c6c67').get_result()
print(session)

authenticator = IAMAuthenticator('j0kSWqlXptEXfuGqkuvZFLSxmMvfV7aHMYiAf7uze0x4')
text_to_speech = TextToSpeechV1(
    authenticator=authenticator
)

text_to_speech.set_service_url('https://api.us-south.text-to-speech.watson.cloud.ibm.com/instances/7c2c7d18-8cc8-407f-b871-37f749ba9c5d')


#authenticator = IAMAuthenticator('PXtrxiU-I5ny32L3k8HPcmzwPvb4bdJFcslA3oIG-48J')
#personality_insights = PersonalityInsightsV3(
#    version='2017-10-13',
#    authenticator=authenticator
#)
#personality_insights.set_disable_ssl_verification(False)
#personality_insights.set_service_url('https://api.us-south.personality-insights.watson.cloud.ibm.com/instances/68f2775d-8ee5-422f-b4a9-7d7d01168f43')

{'session_id': '921cda74-b626-4052-a2d5-a5426a2b0c5d'}


In [7]:
def identLanguage(mensaje):
    language = language_translator.identify(
        mensaje).get_result()
    r= json.dumps(language, indent=2)
    rp= json.loads(r)
    return rp["languages"][0]["language"]

def traducir(mensaje):
    leng=identLanguage(mensaje)
    print("LENguaje Broma ->" ,leng)
    if(leng == "fr" or leng=="ca" or leng=="en"):
        print("Idioma: ",leng)
        translation = language_translator.translate(
        text=mensaje,
        model_id=leng+"-es").get_result()
        r= json.dumps(translation, indent=2)
        rp= json.loads(r)
        return rp["translations"][0]["translation"],leng
    else:
        return mensaje,"es"
    
def traducir2(mensaje,leng):
    print("Lenguaje +",leng)
    if(leng =="fr" or leng=="ca" or leng=="en"):
        print("IAT:",leng)
        translation = language_translator.translate(
        text=mensaje,
        model_id="es-"+leng).get_result()
        r= json.dumps(translation, indent=2)
        rp= json.loads(r)
        return rp["translations"][0]["translation"]
    else:
        return mensaje

def sendMessageIBM(mensaje):
    print(mensaje)
    trad=traducir(mensaje)
    message = assistant.message('ac28a7b9-f962-4376-a98c-1b0a8c5c6c67',
        session["session_id"],
        input={'text': trad[0]},
        context={
            'metadata': {
                'deployment': 'myDeployment'
            }
        }).get_result()
    r=json.dumps(message, indent=2)
    rp=json.loads(r)
    return traducir2(rp['output']['generic'][0]['text'],trad[1])

In [ ]:
#!pip install twilio
from twilio.rest import Client
from flask import Flask, request
import json

app = Flask(__name__)
account_sid = 'AC1c81ac51c252ba506fbe618f742c798b'
auth_token = 'd4a3a3f6aa632452db92aa0faefcb3c4'
client = Client(account_sid, auth_token)

def enviar(mensaje):
    message = client.messages.create(
                              body=mensaje,
                              from_='whatsapp:+14155238886',
                              to='whatsapp:+593988815578'
                          )
    return message


def enviar_foto(url):
    message = client.messages.create(
                              body="Foto",
                              media_url=url,
                              from_='whatsapp:+14155238886',
                              to='whatsapp:+593988815578'
                          )
    return message


@app.route("/", methods=['GET', 'POST'])
def receive_message():
    tt=''
    if request.method == 'GET':
        print('Inicio')
        mensaje = 'Mensaje Inicio APP'
        message = enviar(mensaje)
        tt = message.sid
    else:
        num = request.form['From']
        texto = request.form['Body']
        try:
            texto_respuesta=sendMessageIBM(texto)
        except KeyError:
            texto_respuesta="No le entendido"
        if(texto_respuesta == "Estos son los locales para"):
            texto_respuesta+= " " +texto+"\n"
            texto_respuesta+= getLocales(texto)
        if(texto_respuesta == "Datos de"):
            texto_respuesta+= " " +texto+"\n"
            texto_respuesta+= getComis(texto)
        if(texto_respuesta== "Según sus especificaciones puedes elegir entre estas tres opciones:"):
            texto_respuesta+= " " +texto+"\n"
            ar=texto.split(" ")
            ar=[float(i) for i in ar]
            texto_respuesta+= getComidas(ar)
        if(texto_respuesta=="Los resultados están basados en el análisis con la base de datos:"):
            print(texto)
            if("grasa" in texto and "bajo" in texto):
                texto_respuesta+= "\n"+getRecomendacion(4,"ASC")
            if("grasa" in texto and "alto" in texto):
                texto_respuesta+= "\n"+getRecomendacion(4,"DESC")
            if("calorias" in texto and "bajo" in texto):
                texto_respuesta+= "\n"+getRecomendacion(0,"ASC")
            if("calorias" in texto and "alto" in texto):
                texto_respuesta+= "\n"+getRecomendacion(0,"DESC")
            if("carbohidratos" in texto and "bajo" in texto):
                texto_respuesta+= "\n"+getRecomendacion(1,"ASC")
            if("carbohidratos" in texto and "alto" in texto):
                texto_respuesta+= "\n"+getRecomendacion(1,"DESC")
            if("proteinas" in texto and "bajo" in texto):
                texto_respuesta+= "\n"+getRecomendacion(3,"ASC")
            if("proteinas" in texto and "alto" in texto):
                texto_respuesta+= "\n"+getRecomendacion(3,"DESC")
        message = enviar(texto_respuesta)
        if("McCombos" in texto_respuesta):
            enviar_foto("https://d25dk4h1q4vl9b.cloudfront.net/media/images/promotion-pill/5f2c52e17dd2c5.59193380_BANNER-WEB-GRAND-1243X643.jpg")
        elif("McFlurry" in texto_respuesta):
            enviar_foto("https://d25dk4h1q4vl9b.cloudfront.net/media/images/promotion-pill/5efe4e746edb06.53028580_BANNER-WEB_McFlurry-KitKat.jpg")
        elif("NUEVA Cajita Feliz" in texto_respuesta):
            enviar_foto("https://d25dk4h1q4vl9b.cloudfront.net/media/images/promotion-pill/5efbdd66441150.76999913_1423x623_espaniol___%20(2)%20(1).jpg")
        tt = message.sid
    return tt
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


hola
LENguaje Broma -> es
Lenguaje + es


127.0.0.1 - - [10/Aug/2020 23:34:05] "POST / HTTP/1.1" 200 -
